In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("test").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/07 11:41:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [9]:
from pathlib import Path

import pandas as pd

In [3]:
from pyspark.sql import types as T

In [19]:
yellow_schema = T.StructType(
    [
        T.StructField("VendorID", T.IntegerType(), True),
        T.StructField("tpep_pickup_datetime", T.TimestampType(), True),
        T.StructField("tpep_dropoff_datetime", T.TimestampType(), True),
        T.StructField("passenger_count", T.IntegerType(), True),
        T.StructField("trip_distance", T.FloatType(), True),
        T.StructField("RatecodeID", T.IntegerType(), True),
        T.StructField("store_and_fwd_flag", T.StringType(), True),
        T.StructField("PULocationID", T.IntegerType(), True),
        T.StructField("DOLocationID", T.IntegerType(), True),
        T.StructField("payment_type", T.IntegerType(), True),
        T.StructField("fare_amount", T.FloatType(), True),
        T.StructField("extra", T.FloatType(), True),
        T.StructField("mta_tax", T.FloatType(), True),
        T.StructField("tip_amount", T.FloatType(), True),
        T.StructField("tolls_amount", T.FloatType(), True),
        T.StructField("improvement_surcharge", T.FloatType(), True),
        T.StructField("total_amount", T.FloatType(), True),
        T.StructField("congestion_surcharge", T.FloatType(), True),
    ]
)

green_schema = T.StructType(
    [
        T.StructField("VendorID", T.IntegerType(), True),
        T.StructField("lpep_pickup_datetime", T.TimestampType(), True),
        T.StructField("lpep_dropoff_datetime", T.TimestampType(), True),
        T.StructField("store_and_fwd_flag", T.StringType(), True),
        T.StructField("RatecodeID", T.IntegerType(), True),
        T.StructField("PULocationID", T.IntegerType(), True),
        T.StructField("DOLocationID", T.IntegerType(), True),
        T.StructField("passenger_count", T.IntegerType(), True),
        T.StructField("trip_distance", T.FloatType(), True),
        T.StructField("fare_amount", T.FloatType(), True),
        T.StructField("extra", T.FloatType(), True),
        T.StructField("mta_tax", T.FloatType(), True),
        T.StructField("tip_amount", T.FloatType(), True),
        T.StructField("tolls_amount", T.FloatType(), True),
        T.StructField("ehail_fee", T.FloatType(), True),
        T.StructField("improvement_surcharge", T.FloatType(), True),
        T.StructField("total_amount", T.FloatType(), True),
        T.StructField("payment_type", T.IntegerType(), True),
        T.StructField("trip_type", T.IntegerType(), True),
        T.StructField("congestion_surcharge", T.FloatType(), True),
    ]
)

In [ ]:
df_yellow = pd.read_csv(
    "data/raw/yellow/2021/01/yellow_tripdata_2021-01.csv.gz", nrows=1000
)
df_green = pd.read_csv(
    "data/raw/green/2021/01/green_tripdata_2021-01.csv.gz", nrows=1000
)

In [97]:
services = {"yellow": yellow_schema, "green": green_schema}
for service, schema in services.items():
    for input_path in Path(f"data/raw/{service}").rglob("*.csv.gz"):
        output_path = Path("data/pq") / input_path.parent.relative_to("data/raw")
        output_path.parent.mkdir(parents=True, exist_ok=True)

        df = (
            spark.read.option("header", "true")
            .schema(schema)
            .csv(str(input_path.parent))
        )

        df.repartition(4).write.parquet(str(output_path))